# Quickstart Notebook: PyIntact Campaign to PhysicsNeMo in Istari

This notebook mirrors the command-line quickstart:
1. Connect to Istari
2. Build campaign cases from a JSON spec
3. Submit many simulation jobs
4. Poll completion
5. Trigger training


In [ ]:
%pip install -q -r ../requirements.txt


In [ ]:
from pathlib import Path
from istari_client import get_client
from pyintact.campaign_utils import load_json, generate_cases, make_job_parameters, normalize_status

spec_path = Path('../use-cases/many-pyintact-to-nemo/campaign_spec.example.json')
spec = load_json(spec_path)
cases = generate_cases(spec)
len(cases), cases[0]['case_id']


In [ ]:
client = get_client()
user = client.get_current_user()
user.email


In [ ]:
# Preview first payload before submitting
preview = make_job_parameters(spec, cases[0])
preview


In [ ]:
# Set SUBMIT = True when ready
SUBMIT = False
submitted = []
if SUBMIT:
    for case in cases:
        job = client.add_job(
            model_id=spec['campaign_root_model_id'],
            function='@istari:run_pyintact_simulation',
            parameters=make_job_parameters(spec, case),
        )
        submitted.append({'case_id': case['case_id'], 'job_id': str(job.id), 'status': 'submitted'})
len(submitted)


In [ ]:
# Optional polling loop
import time

POLL = False
if POLL and submitted:
    remaining = True
    while remaining:
        remaining = False
        for row in submitted:
            if row['status'] in {'succeeded','completed','failed','error','cancelled','canceled','timeout','timed_out'}:
                continue
            job = client.get_job(row['job_id'])
            row['status'] = normalize_status(job)
            if row['status'] not in {'succeeded','completed','failed','error','cancelled','canceled','timeout','timed_out'}:
                remaining = True
        print({s: sum(1 for r in submitted if r['status']==s) for s in sorted(set(r['status'] for r in submitted))})
        if remaining:
            time.sleep(20)


## Next

After dataset assembly in Istari, launch training from CLI:

```bash
python ../nemo/launch_training.py --campaign-root-model-id <campaign_root_model_id> --dataset-job-id <dataset_job_id>
```
